# Imports

In [1]:
import logging

import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

%load_ext autoreload
%autoreload 2

/app/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-1d2324e4-e974-4b47-a91e-69bd0e316088.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4189_Onboard_CCXT_Kraken_periodic_daily_trades_spot'
  hash='ec49e07af'
  # Last commits:
    * ec49e07af vlady    CmTask4189: added new symbols in the kraken universe              (    4 days ago) Mon Apr 17 19:01:52 2023  (HEAD -> CmTask4189_Onboard_CCXT_Kraken_periodic_daily_trades_spot, origin/CmTask4189_Onboard_CCXT_Kraken_periodic_daily_trades_spot)
    * 56c5e0a9e vlady    CmTask4189: gallery update             

# Bare CCXT extractor test

In [116]:
since = None

In [120]:
import ccxt

extractor = ccxt.kraken()

start_timestamp = pd.Timestamp("2021-01-01T00:00:00+00:00")
start_timestamp = hdateti.convert_timestamp_to_unix_epoch(start_timestamp)
currency_pair = "ETH/USDT"

data = extractor.fetch_trades(
    currency_pair,
    since=since or start_timestamp,
)

In [122]:
data[0]

{'id': '306690',
 'order': None,
 'info': ['743.14000',
  '2.80536389',
  '1609509956.2953913',
  'b',
  'm',
  '',
  '306690'],
 'timestamp': 1609509956295,
 'datetime': '2021-01-01T14:05:56.295Z',
 'symbol': 'ETH/USDT',
 'type': 'market',
 'side': 'buy',
 'takerOrMaker': None,
 'price': 743.14,
 'amount': 2.80536389,
 'cost': 2084.7781212146,
 'fee': None,
 'fees': []}

In [119]:
since = data[-1]["timestamp"]

In [95]:
data = pd.DataFrame(data)

In [96]:
data.head()

,id,order,info,timestamp,datetime,symbol,type,side,takerOrMaker,price,amount,cost,fee,fees
0,306690,None,"[743.14000, 2.80536389, 1609509956.2953913, b,...",1609509956295,2021-01-01T14:05:56.295Z,ETH/USDT,market,buy,None,743.14,2.805364,2084.778121,None,[]
1,306691,None,"[743.14000, 0.96000000, 1609509956.2967162, b,...",1609509956296,2021-01-01T14:05:56.296Z,ETH/USDT,market,buy,None,743.14,0.960000,713.414400,None,[]
2,306692,None,"[743.15000, 0.70000000, 1609509956.298092, b, ...",1609509956298,2021-01-01T14:05:56.298Z,ETH/USDT,market,buy,None,743.15,0.700000,520.205000,None,[]
3,306693,None,"[743.19000, 0.63267197, 1609509956.2997396, b,...",1609509956299,2021-01-01T14:05:56.299Z,ETH/USDT,market,buy,None,743.19,0.632672,470.195481,None,[]
4,306694,None,"[740.28000, 0.30168360, 1609510398.675571, b, ...",1609510398675,2021-01-01T14:13:18.675Z,ETH/USDT,limit,buy,None,740.28,0.301684,223.330335,None,[]


In [97]:
data.timestamp.agg(["min", "max"]).apply(pd.Timestamp, unit="ms")

min   2021-01-01 14:05:56.295
max   2021-01-02 08:30:39.047
Name: timestamp, dtype: datetime64[ns]

In [107]:
last_data_id = 1609509956295391408

In [114]:
params = {"fromId": last_data_id}
data = extractor.fetch_trades(currency_pair, since=160950995629539.1408)

In [115]:
data

[]

In [105]:
since = data[-1]["info"][-1]

# CCXT extractor test

In [123]:
import im_v2.ccxt.data.extract.extractor as imvcdexex

exchange_id = "kraken"
contract_type = "spot"
ccxt_extractor = imvcdexex.CcxtExtractor(exchange_id, contract_type)

In [125]:
data_type = "trades"
currency_pair = "ETH/USDT"
converted_currency_pair = currency_pair
start_timestamp = pd.Timestamp("2021-01-01T00:00:00+00:00")
end_timestamp = pd.Timestamp("2021-01-07T23:59:59+00:00")
data = ccxt_extractor.download_data(
    data_type,
    exchange_id,
    converted_currency_pair,
    start_timestamp=start_timestamp,
    end_timestamp=end_timestamp,
    depth=10,
)

In [126]:
data.head()

,timestamp,symbol,side,price,amount,end_download_timestamp
0,1609459218231,ETH/USDT,sell,735.91,0.035530,2023-04-21 16:41:36.288763+00:00
1,1609459326746,ETH/USDT,buy,738.76,0.373670,2023-04-21 16:41:36.288763+00:00
2,1609459326749,ETH/USDT,buy,738.76,0.783000,2023-04-21 16:41:36.288763+00:00
3,1609459387015,ETH/USDT,sell,737.36,1.015312,2023-04-21 16:41:36.288763+00:00
4,1609459387018,ETH/USDT,sell,737.35,1.213466,2023-04-21 16:41:36.288763+00:00


In [30]:
data["timestamp1"] = data["timestamp"].apply(pd.Timestamp, unit="ms")

In [31]:
len(data)

1000

In [127]:
data.timestamp.agg(["min", "max"]).apply(pd.Timestamp, unit="ms")

min   2021-01-01 00:00:18.231
max   2021-01-07 23:59:12.423
Name: timestamp, dtype: datetime64[ns]

In [10]:
data.isna().any().any()

False

# Bulk script data download

In [4]:
%%bash
/app/im_v2/common/data/extract/download_bulk.py \
    --download_mode 'periodic_daily' \
    --downloading_entity 'airflow' \
    --action_tag 'downloaded_1min' \
    --vendor 'ccxt' \
    --start_timestamp '2023-02-01T00:00:00+00:00' \
    --end_timestamp '2023-02-20T00:00:00+00:00' \
    --exchange_id 'kraken' \
    --incremental \
    --universe 'v7.4' \
    --aws_profile 'ck' \
    --data_type 'trades' \
    --data_format 'parquet' \
    --contract_type 'spot' \
    --s3_path 's3://cryptokaizen-data-test/'

INFO: > cmd='/app/im_v2/common/data/extract/download_bulk.py --download_mode periodic_daily --downloading_entity airflow --action_tag downloaded_1min --vendor ccxt --start_timestamp 2023-02-01T00:00:00+00:00 --end_timestamp 2023-02-20T00:00:00+00:00 --exchange_id kraken --incremental --universe v7.4 --aws_profile ck --data_type trades --data_format parquet --contract_type spot --s3_path s3://cryptokaizen-data-test/'
report_memory_usage=False report_cpu_usage=False
INFO: Saving log to file '/app/im_v2/common/data/extract/download_bulk.py.log'
14:53:19 - INFO  dataset_schema_utils.py _get_dataset_schema_file_path:39 Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
14:53:19 - INFO  dataset_schema_utils.py get_dataset_schema:74          Loaded dataset schema version v3
14:53:20 - INFO  dataset_schema_utils.py _get_dataset_schema_file_path:39 Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
14:53:20 - IN

# Read and check data by RawReader

In [5]:
signature = "periodic_daily.airflow.downloaded_1min.parquet.trades.spot.v7_4.ccxt.kraken.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
start_timestamp = pd.Timestamp("2023-02-01T00:00:00+00:00")
end_timestamp = pd.Timestamp("2023-02-20T00:00:00+00:00")
trades = reader.read_data(start_timestamp, end_timestamp)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3
INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [6]:
trades

,timestamp,symbol,side,price,amount,exchange_id,knowledge_timestamp,currency_pair,year,month,day
timestamp,,,,,,,,,,,
2023-02-01 00:11:20.376000+00:00,1675210280376,ADA/USDT,buy,0.389438,15.000000,kraken,2023-04-17 18:53:32.446532+00:00,ADA_USDT,2023,2,1
2023-02-01 00:41:59.149000+00:00,1675212119149,ADA/USDT,buy,0.390383,15.000000,kraken,2023-04-17 18:53:32.446532+00:00,ADA_USDT,2023,2,1
2023-02-01 00:42:25.063000+00:00,1675212145063,ADA/USDT,sell,0.389947,15.000000,kraken,2023-04-17 18:53:32.446532+00:00,ADA_USDT,2023,2,1
2023-02-01 00:42:27.405000+00:00,1675212147405,ADA/USDT,buy,0.390153,15.000000,kraken,2023-04-17 18:53:32.446532+00:00,ADA_USDT,2023,2,1
2023-02-01 00:42:51.891000+00:00,1675212171891,ADA/USDT,buy,0.389907,15.000000,kraken,2023-04-17 18:53:32.446532+00:00,ADA_USDT,2023,2,1
...,...,...,...,...,...,...,...,...,...,...,...
2023-02-03 14:51:33.681000+00:00,1675435893681,XRP/USDT,buy,0.411280,2540.150019,kraken,2023-04-17 18:53:28.350857+00:00,XRP_USDT,2023,2,3
2023-02-03 14:51:33.681000+00:00,1675435893681,XRP/USDT,buy,0.411290,13042.648019,kraken,2023-04-17 18:53:28.350857+00:00,XRP_USDT,2023,2,3
2023-02-03 14:51:33.681000+00:00,1675435893681,XRP/USDT,buy,0.411500,1270.452268,kraken,2023-04-17 18:53:28.350857+00:00,XRP_USDT,2023,2,3


In [7]:
trades.isna().any().any()

False

In [9]:
pd.Series(trades.currency_pair.unique())

0       ADA_USDT
1       APE_USDT
2           ATOM
3      AVAX_USDT
4            BCH
5       BTC_USDT
6       DAI_USDT
7      DOGE_USDT
8       DOT_USDT
9       ETH_USDT
10          LINK
11      LTC_USDT
12    MATIC_USDT
13     SHIB_USDT
14      SOL_USDT
15     USDC_USDT
16           XMR
17      XRP_USDT
dtype: object

In [10]:
a = [
    "ETH_USDT",
    "BTC_USDT",
    "APE_USDT",
    "MATIC_USDT",
    "DOT_USDT",
    "XRP_USDT",
    "DOGE_USDT",
    "SOL_USDT",
    "USDC_USDT",
    "ADA_USDT",
    "LTC_USDT",
    "SHIB_USDT",
    "AVAX_USDT",
    "DAI_USDT",
    "LINK/USDT",
    "ATOM/USDT",
    "XMR/USDT",
    "BCH/USDT",
]

In [19]:
unique_currency_pair = pd.Series(trades.currency_pair.unique())

In [21]:
unique_currency_pair[~unique_currency_pair.isin(a)]

2     ATOM
4      BCH
10    LINK
16     XMR
dtype: object

# QA  exploratory analysis

In [73]:
mode = "download"
vendor_name = "CCXT"
version = "v7.4"
universe = imvcounun.get_vendor_universe(vendor_name, mode, version=version)
universe_list = universe[exchange_id]
qa_check_list = [
    imvcdqqach.NaNChecks(),
    imvcdqqach.FullUniversePresentCheck(universe_list),
]
dataset_validator1 = imvcdqdava.DataFrameDatasetValidator(qa_check_list)

In [74]:
dataset_validator1.run_all_checks([trades])

INFO  Running all QA checks:
INFO  	NaNChecks: PASSED


AssertionError: 
################################################################################
	FullUniversePresentCheck: FAILED: Found missing symbols in dataset:
	{'MATIC_USDT', 'APE_USDT', 'AVAX_USDT', 'SOL_USDT'}
################################################################################
